## Lab 2
### Aaron Sigman
There are two types of indices: broadband and narrowband as shown in this paper: "Evaluation of Broadband and Narrowband Vegetation Indices for the Identification of Archaeological Crop Marks". Implement 4 other broadbands vegetation indices for Logan, UT as set up in this notebook. Do you note a difference in response between the NDVI and EVI vs the ones you implemented in agricultural areas, bare soils, and water? Please explain it in a notebook.

- Determine what other indicies exist. 

Indicies used in the lab: NDVI, EVI, NDWI, NDBSI, NDSI \
Broadband indicies to pick from: Green NDVI, SR, MSR, MTVI2, RDVI, IRG, PVI, RVI, TSAVI, MSAVI, ARVI, GEMI, SARVI, OSAVI, SR x NDVI \

- Select 4 bands to use. 
    - Green NDVI (Green Normalized Difference Vegetation Index) 
    
    GNDVI = (p$_{NIR}$-p$_{green}$)/(p$_{NIR}$+p$_{green}$)
    - MSR (Modified Simple Ratio) 
    
    MSR = p$_{red}$/(p$_{NIR}$/p$_{red}$ +1)$^{0.5}$
    - RDVI (Renormalize Difference Vegetation Index) 
    
    RDVI = (p$_{NIR}$ – p$_{red}$)/(p$_{NIR}$ + p$_{red}$)$^{0.5}$
    - MSAVI (Modified Soil Adjusted Vegetation Index) 
    
    MSAVI = [(2p$_{NIR}$+1-[(2p$_{NIR}$+1)$^2$-8(p$_{NIR}$-p$_{red}$)]$^{0.5}$]/2

### Initialize display and earth engine

In [1]:
from IPython.display import Image
import ee
ee.Initialize()

In [2]:
import geemap

### Define a point in Logan, Utah

In [3]:
# Define point at Logan, UT
Lat=41.735210
Long=-111.834862
Point = ee.Geometry.Point([Long, Lat])

### Get Landsat8 imagery
- Get landsat8 imagery using reference location from google earth engine datasets
    - Filter by images that overlap point
    - Filter by selected dates
    - Sort by cloud cover
    - Select the first (least cloudy) image to use
- Select the bands that we are going to use (Bands 1-7)
- Correct the image using the correction factor and offset from google earth engine datasets

In [4]:
landsat8 =ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") #change

image = ee.Image(landsat8
    .filterBounds(Point)
    .filterDate('2021-06-01', '2021-09-01')
    .sort('CLOUD_COVER_LAND')
    .first())

image = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])


image = image.multiply(0.0000275).add(-0.2)

### Create Interactive Map
- Center around the selected point.
- Add the true color image

In [5]:
Map = geemap.Map() # from ipygee
bounds = Point.buffer(10000)
Map.centerObject(bounds)
trueColor = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.4}
Map.addLayer(image, trueColor, name='Landsat8 Image')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
ndvi = image.normalizedDifference(['SR_B5', 'SR_B4'])
evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('SR_B5'),
      'RED': image.select('SR_B4'),
      'BLUE': image.select('SR_B2')
})

In [7]:
vegPalette = ['white','red','yellow', 'green','blue']
Map.addLayer(ndvi, {'min': 0, 'max': 1, 'palette': vegPalette},name = 'NDVI',opacity=1)
Map.addLayer(evi, {'min': 0, 'max': 1, 'palette': vegPalette}, 'EVI',opacity=1)
Map

Map(center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['position', 'transparent_…

### - Green NDVI
- GNDVI = (p$_{NIR}$-p$_{green}$)/(p$_{NIR}$+p$_{green}$)
- Compute normalize difference between NIR and green

Green NDVI is similar to NDVI except that instead of the red spectrum it measures the green spectrum. This is an indicator of the photosynthetic activity of the vegetation cover; it is most often used in assessing the moisture content and nitrogen concentration in plant leaves according to multispectral data which do not have an extreme red channel. Compared to the NDVI index, it is more sensitive to chlorophyll concentration. It is used in assessing depressed and aged vegetation. 

In [8]:
gndvi = image.normalizedDifference(['SR_B5', 'SR_B3'])
Map.addLayer(gndvi, {'min': 0, 'max': 1.0, 'palette': vegPalette},name = 'Green NDVI',opacity=1)
Map

Map(center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['position', 'transparent_…

#### Results
Green NDVI and Red NDVI both exclude water well. EVI doesn't seem to select water very well. The values for green ndvi don't go as high for the fields. Since GNDVI is more sensitive to chlorophyll concentration, it can tell more about plant health than plant density like red NDVI. EVI is used for detecting vegetation in areas with a high leaf area index, essentially removing some of the effects of soil, focusing on the vegetation. EVI seems to be more stricty selecting areas of healthy vegetation.

### Modified Simple Ratio
- MSR = p$_{red}$/(p$_{NIR}$/p$_{red}$ +1)$^{0.5}$

In [9]:
msr = image.expression(
    'RED / ((NIR / RED) + 1)**(1/2)', {
      'NIR': image.select('SR_B5'),
      'RED': image.select('SR_B4')
})

In [17]:
vegInv=['blue','green','yellow','red','white']
Palette2=['green','white']
Map.addLayer(msr, {'min': 0, 'max': .2, 'palette': vegInv}, 'MSR',opacity=1)
Map

Map(bottom=780653.0, center=[41.76298940381476, -111.86090469360353], controls=(WidgetControl(options=['positi…

#### Results
MSR doesn't seem to be the best tool for characterizing water or plants, as these areas sometimes look to fall within the same ranges. MSR does seem to do a decent job at characterizing more urban areas.

### Renormalize Difference Vegetation Index
- RDVI = (p$_{NIR}$ – p$_{red}$)/(p$_{NIR}$ + p$_{red}$)$^{0.5}$

In [11]:
rdvi = image.expression(
    '(NIR - RED)/((NIR + RED)**(1/2))', {
      'NIR': image.select('SR_B5'),
      'RED': image.select('SR_B4')
})

In [12]:
Map.addLayer(rdvi, {'min': 0, 'max': 1, 'palette': vegPalette}, 'RDVI',opacity=1)
Map

Map(center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['position', 'transparent_…

### Modified Soil Adjusted Vegetation Index
- MSAVI = [(2p$_{NIR}$+1-[(2p$_{NIR}$+1)$^2$-8(p$_{NIR}$-p$_{red}$)]$^{0.5}$]/2

In [13]:
msavi = image.expression(
    '(2*NIR+1 - ((((2*NIR+1)**2)-(8*(NIR-RED)))**(0.5)))/2', {
      'NIR': image.select('SR_B5'),
      'RED': image.select('SR_B4')
})

In [14]:
Map.addLayer(msavi, {'min': 0, 'max': 1, 'palette': vegPalette}, 'MSAVI',opacity=1)
Map

Map(center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['position', 'transparent_…